
<div style="width: 100%; clear: both;">
    <div style="float: left; width: 50%;">
       <img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg", align="left">
    </div>
</div>

<div style="float: right; width: 50%;">
    <p style="margin: 0; padding-top: 22px; text-align:right;">M1.771 · Privacidad</p>
    <p style="margin: 0; text-align:right;">Máster Universitario en Ciberseguretat y Privacidad</p>
    <p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>

</div>
<div style="width: 100%; clear: both;">
<div style="width:100%;">&nbsp;</div>


Privacidad
============================

---

Ejercicios resueltos - Privacidad diferencial
-----------------------------------------------------

---

### Instrucciones de uso

Este *notebook* contiene un conjunto de ejercicios sobre privacidad diferencial
que os pueden servir para practicar los conceptos que se explican en los módulos
teóricos.

Os recomendamos que intentéis hacer estos problemas vosotros mismos y que, una
vez realizados, comparéis vuestra solución con la que os proponemos. No dudéis
en dirigiros al foro del aula para consultar todas las cuestiones que surjan a raíz de la resolución de estos
ejercicios o bien de las soluciones dadas.

Para hacer las actividades propuestas, tendríais que estar familiarizados con
las librerías de Python `Pandas` y `NumPy`. Si no conocéis el funcionamiento
de estas librerías, os recomendamos que reviséis primero las unidades de los
*notebooks* de la asignatura Cimientos de programación que encontraréis en el aula.

El uso de varias funciones de las librerías `Pandas` o `NumPy` os puede ser muy
útil en la resolución de algunos ejercicios. Como ayuda, algunas preguntas
sugieren el uso de alguna de estas funciones en una **nota**. Sois libres de decidir si seguir o no estas
recomendaciones. De hecho, puede haber más de una manera de solucionar los ejercicios, y la que os
sugerimos puede no ser la mejor.


### Introducción

En este *notebook* pondremos en práctica algunas ideas básicas de la aplicación
de privacidad diferencial. En general, trabajaremos con ejemplos muy sencillos y simularemos
el hecho de hacer una consulta a una base de datos, simplemente como
un llamamiento a una función en Python que recibe como parámetro la base de datos como
un `DataFrame` de `Pandas`. 


Las dos celdas de código siguientes permiten activar las alertas de estilo para el
código de este *notebook*. Si queréis comprobar que vuestro código sigue la guía
de estilo de Python, podéis descomentarlas y ejecutarlas. A partir de entonces,
cualquier otra celda de código que se ejecute pasará una revisión de estilo; también
se mostrarán alertas que informarán de las líneas de código que no cumplen con
la guía y de cuál es el problema detectado.


In [1]:
# this needs the packages: ppycodestyle, pycodestyle_magic, flake8
#%load_ext pycodestyle_magic

In [2]:
#%pycodestyle_on



# Privacidad diferencial

Se ha hecho una encuesta sobre quién ha leído *El Quijote*. El resultado de la
encuesta incluye datos sobre todos los sujetos que la han contestado y proporciona
información sobre si el sujeto ha leído o no la novela. Por motivos
que se escapan a nuestra compresión se considera la información sobre si un sujeto
ha leído o no *El Quijote* como muy sensible y se quiere mantener secreta a toda
costa.

Los resultados de esta encuesta los encontramos en el fichero `data/quixote_today.csv`.

Nuestra base de datos contendrá la información de este fichero, pero
limitaremos el acceso. Solo se podrá acceder mediante una API
pública, que incluirá una función `query` que devolverá el número de sujetos de la
base de datos que SÍ que han leído la novela.

Asumimos que los usuarios externos y posibles atacantes solo tienen acceso a la
base de datos usando una función `query`. Esta situación puede parecer
relativamente segura respecto a la privacidad de los sujetos de la base de datos.
También puede parecer que un atacante no podría obtener información sobre si un
sujeto concreto ha leído o no *El Quijote*. Como veremos, y como ya debéis de sospechar,
esto no es del todo cierto, puesto que dependiendo de la situación en la cual nos encontramos se
podría llegar a filtrar información confidencial.


**Ejercicio 1**

Cargad el fichero `data/quixote_today.csv` como un DataFrame de `Pandas`
en la variable `db1`.


In [3]:
import pandas as pd


def load_db(filename: str) -> pd.DataFrame:
    return pd.read_csv(filename)


db1 = load_db("data/quixote_today.csv")

**Ejercicio 2**

Como ya hemos comentado, simularemos que `db1` es la base de datos y que la única manera
que tienen los usuarios externos de acceder es mediante la API pública.
Esta API proporciona una función `query` que devuelve el número de sujetos
que sí que han leído *El Quijote*.

Implementad esta función `query` que recibe como parámetro una base de datos, es decir,
un DataFrame como `db1`, y que devuelve el número de sujetos que han leído *El Quijote*. Este
número es el que se enviará al usuario como respuesta.

Probad la función con la base de datos `db1` y mirad qué devuelve.


In [4]:
def query(db: pd.DataFrame) -> int:
    return db["quixote"].sum()


query(db1)

31

## Posible ataque a la privacidad

Suponemos la situación siguiente: acabamos de ver que apenas
han encuestado a nuestro vecino Richard Flores y sabemos que sus respuestas se
guardan automáticamente a la base de datos. Es decir, podemos hacer una consulta justo
después de que se haya hecho la encuesta a Richard Flores.

Con esto asumimos que podemos hacer dos consultas: una justo antes de que se haya
encuestado a Richard Flores y otra después.

En el contexto de la privacidad diferencial esto puede ser equivalente a tener dos bases de datos:
una que contiene el registro correspondiente a Richard Flores y la otra que es
exactamente igual al anterior pero sin este registro.

Por eso usaremos las bases de datos `data/quixote_today.csv` y `data/quixote_yesterday.csv`.
De forma que:
* `quixote_today.csv` contiene todos el registros, incluyendo el de Richard Flores.
* `quixote_yesterday.csv` contiene los mismos registros sin el de Richard Flores.


**Ejercicio 3**

Cargad el fichero `data/quixote_yesterday.csv` y guardad el DataFrame resultante como
`bd2`.

Probad a hacer la consulta `query` sobre esta base de datos.

In [5]:
db2 = load_db("data/quixote_yesterday.csv")

query(db2)

31

**Ejercicio 4**

Comparando las dos consultas hechas a `bd1` y `bd2`, podemos saber si Richard Flores
ha leído *El Quijote*?

> *Respuesta*: sabemos que `db1` tiene precisamente un solo registro más que `db2`; además,
> también sabemos que este registro corresponde a Richard Flores. Si comparamos la ejecución
> de la función `query` en los dos casos, podemos comparar el número de sujetos que
> han leído *El Quijote* en los dos casos. Es decir, si Richard Flores ha leído
> la novela, la diferencia entre `db1` y `db2` será 1, mientras que, si no la ha
> leído, será 0.


In [6]:
diff = query(db1) - query(db2)
if diff == 1:
    print("Richard Flores has read the Quixote")
else:
    print("Richard Flores has not read the Quixote")

Richard Flores has not read the Quixote


# Mecanismo de Laplace

Ahora aplicaremos el mecanismo de Laplace para proteger el acceso a la base de datos
e intentar evitar el ataque que hemos visto anteriormente. Es decir, haremos
una versión de la función `query` que cumpla privacidad diferencial, y para ello
usaremos el mecanismo de Laplace.

Recordemos que, en privacidad diferencial, el mecanismo de Laplace se puede aplicar a funciones
numéricas del tipo $f: {\cal D} \to \mathbb{R}^k$, es decir, una función que
aplicada a la base de datos $X \in {\cal D}$ devuelve $k$ números reales (en
este caso denotamos como ${\cal D}$ el conjunto de todas las posibles bases de datos).

Para definir el mecanismo de Laplace, hace falta que conozcamos el concepto de *sensibilidad
global* y cómo se calcula la distribución de Laplace.

La **sensitividad global** o $l_1$-_sensitivity_ de una función $f: {\cal D} \to
\mathbb{R}^k$ se define como:

$$GS_f = \max_{X_1, X_2 \in {\cal D}, \ d(X_1, X_2) = 1} \lVert f(X_1) - f(X_2) \rVert_{1}$$

donde $\lVert \cdot \rVert_1$ es la norma $L_1$. Dado un vector
$x \in \mathbb{R}^M$, $\lVert x \rVert_1 = \sum_{y=1}^M | x_y |$, es decir, la suma
del valor absoluto de todos los elementos del vector. 

En este caso $X_1$ y $X_2$ pueden ser cualquier par de versiones de la base de datos
que difieren en un solo registro. Esto se denota como $d(X_1, X_2) = 1$.

Así pues, de manera simplificada podemos definir el mecanismo de Laplace para
una función $f: {\cal D} \to \mathbb{R}^k$ como:

$$M_L(x,f,\epsilon) = f(x) + (L_1, \ldots, L_k)$$

donde $L_i$ son variables aleatorias obtenidas a partir de una distribución de Laplace
$Lap(GS_f/\epsilon)$. 

Recordemos que $Lap(b)$ denota la distribución de Laplace con escala $b$ (y centrada
en el 0) que tiene la función de densidad de probabilidades:

$$Lap(x,b) = \frac{1}{2b}\exp(-\frac{|x|}{b})$$



**Ejercicio 5**

Calculad cuál es la sensibilidad global de la función `query` que hemos definido
anteriormente y guardad su valor en una variable `GS_query`.

> *Respuesta*: la sensibilidad global de `query` es 1. Sabemos que la diferencia de
> la respuesta a una función `query` hecha a dos bases de datos que difieren en un solo
> elemento será 0 o 1. Así pues, la diferencia máxima es 1.

In [7]:
GS_query = 1


**Ejercicio 6** 

Definid una nueva función `query_laplace` que use la función anterior
`query` y devuelva el resultado de aplicar el mecanismo de Laplace para la función
`query`. La nueva función `query_laplace` recibirá como parámetro una base de datos
(`DataFrame`) y el valor $\epsilon$. Para ser coherentes con la interpretación
del resultado haremos que devuelva un entero.

> **Nota**: podéis usar la función [`numpy.random.Generator.laplace`](https://numpy.org/doc/stable/reference/random/generated/numpy.random.Generator.laplace.html).


In [8]:
import numpy as np


def query_laplace(db: pd.DataFrame, dp_epsilon: float) -> int:
    lap = np.random.default_rng().laplace(0, GS_query / dp_epsilon, 1)[0]
    return int(query(db) + lap)

**Ejercicio 7**

Ejecutad la función `query_laplace` con `db1` y `db2` y comparad el resultado con
el de aplicar la función `query`. ¿Qué pasa con $\epsilon = 0.1$ y con $\epsilon =
10$? Debido a la aleatoriedad de `query_replace`, se recomienda que ejecutéis la función
diversas veces.

¿Ahora podemos saber si Richard Flores ha leído *El Quijote* si se usa la función
`query_laplace`?

In [9]:
print("Epsilon = 0.1:")
print(f"  DB1: query {query(db1)}, query_laplace {query_laplace(db1, 0.1)}")
print(f"  DB2: query {query(db2)}, query_laplace {query_laplace(db2, 0.1)}")
print("Epsilon = 10:")
print(f"  DB1: query {query(db1)}, query_laplace {query_laplace(db1, 10)}")
print(f"  DB2: query {query(db2)}, query_laplace {query_laplace(db2, 10)}")

Epsilon = 0.1:
  DB1: query 31, query_laplace 36
  DB2: query 31, query_laplace 32
Epsilon = 10:
  DB1: query 31, query_laplace 30
  DB2: query 31, query_laplace 30


> *Respuesta*: vemos que en el caso de $\epsilon = 0.1$ la respuesta devuelta por
> las funciones `query` y `query_laplace` es bastante diferente. Si probamos de hacer
> el ataque descrito en el ejercicio 4, obtenemos resultados poco coherentes, puesto que es
> muy posible que esta diferencia no sea ni 0 ni 1, que son los valores
> que esperaríamos obtener.
>
> Recordad que el valor concreto dependerá de la aleatoriedad de la ejecución
> (probad de ejecutarlo unas cuantas veces).
>
> También podemos observar que el valor devuelto es bastante diferente al original
> (que es 31). Esto nos está mostrando que hay un nivel de privacidad muy
> alto a costa de introducir una pérdida de información muy grande.
>
> Por el contrario, cuando probamos con $\epsilon = 10$ vemos que las repuestas entre
> `query_laplace` y `query` son muy parecidas. De hecho, si ejecutáis muchas veces
> la función podéis encontrar que en muchos casos devuelven el mismo valor. Esto nos
> muestra que el nivel de privacidad es muy bajo (en muchos casos
> el ataque del ejercicio 4 se puede hacer con éxito), y, por el contrario, que tenemos una pérdida
> de información muy baja.
>
> Para verlo más claro podemos comparar la media de la diferencia entre
> ejecutar la función `query` y la `query_laplace` tanto en `db1` como en `db2` en
> valor absoluto para los dos valores de $\epsilon$:

In [10]:
def compare_results(epsilon):
    q1 = q2 = 0
    for i in range(1000):
        q1 += abs(query(db1) - query_laplace(db1, epsilon))
        q2 += abs(query(db2) - query_laplace(db2, epsilon))
    return q1/100, q2/100

print(f" epsilon=0.1: {compare_results(0.1)}")
print(f" epsilon=10: {compare_results(10)}")

 epsilon=0.1: (100.11, 104.3)
 epsilon=10: (4.93, 4.84)


**Ejercicio 8**

Para practicar un poco lo que hemos visto, ahora queremos hacer lo mismo pero con una
función `query2`, que devolverá el número de sujetos que NO han leído *El Quijote*
y su edad media. 

Para ello definiremos una nueva función que recibe como parámetro una base de datos
(un `DataFrame`) y devuelve una tupla de dos números. El primero es el número
de personas que NO han leído la novela, y el segundo, su edad media (media aritmética de los que no han leído El Quijote).


In [11]:
from typing import Tuple


def query2(db: pd.DataFrame) -> Tuple[int, int]:
    quixote = db[db["quixote"] == 0].shape[0]
    avg_age = db[db['quixote'] == 0]["age"].mean()
    return (quixote, avg_age)


print(f"DB1: {query2(db1)}, DB2: {query2(db2)}")

DB1: (69, 40.44927536231884), DB2: (68, 40.411764705882355)


**Ejercicio 9**

Comparad la ejecución de `query2` con `db1` y `db2`. ¿Ahora podemos saber si
Richard Flores ha leído o no *El Quijote*? ¿Tenemos información sobre su edad?

> *Respuesta*: siguiendo con el razonamiento del ejercicio 4, podemos comparar
> la ejecución de la función `query2` en `db1` y en `db2`. Del mismo modo, podemos
> saber si Richard Flores ha leído o no *El Quijote* (notad que en este caso
> la función devuelve el número de los que NO lo han leído).
>
> En cuanto a la edad, nos encontramos que podemos saber la edad exacta de
> Richard Flores, puesto que tenemos la media de edad de los que no han leído la
> novela, así como el número de sujetos que no lo han leído con Richard Flores y sin él.
> Si denotamos como $\mu_2$ la media de los que no la han leído en `db2`, tenemos que:
>
> $$ \mu_2 = \frac{\sum x_i}{N_1} $$
>
> donde $x_i$ son las edades de todos los sujetos que no han leído *El Quijote* y
> $N_1$ el número de sujetos que no lo han leído en `db2`
> (esto excluye a Richard Flores). En el caso de `db1` tenemos:
>
> $$ \mu_1 = \frac{(\sum x_i) + y}{N_2} $$
>
> donde ahora $y$ corresponde a la edad de Richard Flores y $N_2$ es el número de
> sujetos que no han leído *El Quijote* en `db2` (presumiblemente $N_2 = N_1 + 1$).
> Jugando un poco con estas dos fórmulas podemos encontrar el valor de $y$.
>
> Es importante notar que podemos calcular la edad de Richard Flores porque no ha
> leído *El Quijote*, es decir, porque $N_2 = N_1 + 1$. En caso contrario, si
> tuvieramos que $N_2 = N_1$, sabríamos que Richard Flores sí que ha leído la
> novela, pero no podríamos saber su edad.

In [12]:
q1 = query2(db1)
q2 = query2(db2)

diff = q1[0] - q2[0]
if diff == 0:
    print("Richard Flores has read Don Quixote")
else:
    print("Richard Flores has not read Don Quixote")

print(f"Age of Richard Flores: {q1[0]*q1[1] - q2[0]*q2[1]}")

Richard Flores has not read Don Quixote
Age of Richard Flores: 43.0


**Ejercicio 10**

Para evitar el problema anterior, utilizaremos privacidad diferencial.
Definid una función `query2_laplace` que devuelva el resultado de aplicar el mecanismo de Laplace a la
función `query2`. La nueva función `query2_laplace` recibirá como parámetro una base
de datos (un `DataFrame` de` Pandas`) y el valor $\epsilon$.

Tened en cuenta que para poder determinar la sensibilidad global de la función
`query2`, el resultado de la misma debe estar acotado. Por ello asumiremos que
el atributo `age` puede ser como mínimo 0 y como máximo 100.

Probad ahora de ejecutar la función `query2_laplace` en `db1` y en `db2` para $\epsilon = 0.1$, $\epsilon =
10$ i $\epsilon=100$.

¿Ahora podemos saber si Richard Flores ha leído *El Quijote* o qué edad tiene, tal y
como lo hemos hecho en el ejercicio 9?


> *Respuesta*: primero tenemos que calcular la sensitividad global de `query2`.
> Esta función devuelve una tupla $(N, e)$, donde $N$ es el número de sujetos
> que no han leído *El Quijote*, y $e$, su edad media. Como hemos visto, la
> sensibilidad global se basa en la diferencia máxima que puede haber entre dos
> consultas hechas a dos bases de datos que solo difieren en un elemento.
>
> En este caso, ya hemos visto que la diferencia máxima para $N$ sería 1
> (ejercicio 5). En el caso de la edad mediana, la diferencia más grande depende de los
> valores máximo y mínimo de edad posibles. Suponemos el caso en el que tenemos DB1 con 1
> registro y DB2 con 2. Si la edad del registro en DB1 es 0 y el nuevo registro de
> DB2 es 100, tenemos una diferencia entre DB1 y DB2 de 50 (asumiendo que los dos
> registros no hayan leído *El Quijote*). En general, la diferencia máxima en
> la media de edad será $50$.
>
> Con esto podemos decir que la sensibilidad global de `query2` es de $51$. Recordad
> que, en el supuesto de que la respuesta sea un vector, hay que dar la suma de elementos
> en valor absoluto de la diferencia de las respuestas.

In [13]:
# global sensitivity of function `query2`:
# in case you wonder, the l1 norm can be computed with numpy as:
# np.linalg.norm((1,50), ord=1)
GS_query2 = 51

> *Respuesta*: ahora podemos calcular la función `query2_laplace` usando el
> mecanismo de Laplace.

In [14]:
def query2_laplace(db: pd.DataFrame, dp_epsilon: float) -> Tuple[int, int]:
    l_num = np.random.default_rng().laplace(0,GS_query2 / dp_epsilon,1)[0]
    l_age = np.random.default_rng().laplace(0,GS_query2 / dp_epsilon,1)[0]
    q2 = query2(db)
    return int(q2[0] + l_num), int(q2[1] +l_age)

> *Respuesta*: para ver el resultado, y si estamos consiguiendo
> anonimizar la respuesta, calculamos la ejecución de `query2_laplace` sobre `db1`
> y `db2` y miramos si podemos calcular la edad de Richard Flores, tal y como 
> hemos hecho en el ejercicio 10. Todo esto para los diferentes valores de $\epsilon$.
>
> Las respuestas respecto al número de sujetos que no han leído *El Quijote* se
> van acercando a la real (69 en `db1` y 68 en `db2`) a medida que aumentamos
> $\epsilon$. Si lo comparamos con el resultado de `query_laplace`, se puede apreciar que
> ahora hay algo más de ruido para los mismos valores $\epsilon$, puesto que
> la sensibilidad es mayor.
>
> En cuanto a la edad, vemos que a medida que $\epsilon$ crece, el cálculo se acerca
> más a la edad real. La aleatoriedad puede hacer que haya que ejecutarlo varias
> veces para tener una idea del tipo de valor que obtendremos en cada caso.
> Para hacerlo de manera rápida (aunque quizás no muy rigurosa), hemos hecho una
> función `age_estimation` que se ejecuta mil veces y devuelve la media de todas las ejecuciones.

In [15]:
def age_estimation(epsilon: float) -> float:
    avg = 0
    n_loops = 1000
    for i in range(n_loops):
        q1 = query2_laplace(db1, epsilon)
        q2 = query2_laplace(db2, epsilon)
        avg += q1[0]*q1[1] - q2[0]*q2[1]
    return avg/n_loops

for epsilon in [0.1, 10, 100]:
    q1 = query2_laplace(db1, epsilon)
    q2 = query2_laplace(db2, epsilon)
    print(f"*** epsilon = {epsilon}")
    print(f" DB1: {q1}")
    print(f" DB2: {q2}")

    print(f"Age of Richard Flores: {age_estimation(epsilon)}")



*** epsilon = 0.1
 DB1: (707, -491)
 DB2: (706, -83)
Age of Richard Flores: -30375.75
*** epsilon = 10
 DB1: (72, 37)
 DB2: (67, 63)
Age of Richard Flores: 28.754
*** epsilon = 100
 DB1: (68, 40)
 DB2: (68, 40)
Age of Richard Flores: 36.77
